In [16]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from keras import backend as K

In [28]:
model = Sequential()
model.add(Dense(5, activation='linear', input_dim = 3))
model.add(Dense(3, activation='linear'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 5)                 20        
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 18        
Total params: 38
Trainable params: 38
Non-trainable params: 0
_________________________________________________________________


In [29]:
sgd = SGD(lr=0.001, decay=1e-8)
model.compile(loss='mean_squared_error', optimizer=sgd)

In [30]:
x_train = np.random.random(size=(100,3))
y_train = x_train

In [31]:
model.fit(x_train, y_train, epochs=100)

Epoch 1/100
100/100 [==============================] - 0s 1ms/step - loss: 0.3104
Epoch 2/100
100/100 [==============================] - 0s 40us/step - loss: 0.3072
Epoch 3/100
100/100 [==============================] - 0s 40us/step - loss: 0.3044
Epoch 4/100
100/100 [==============================] - 0s 40us/step - loss: 0.3016
Epoch 5/100
100/100 [==============================] - 0s 50us/step - loss: 0.2990
Epoch 6/100
100/100 [==============================] - 0s 50us/step - loss: 0.2961
Epoch 7/100
100/100 [==============================] - 0s 40us/step - loss: 0.2936
Epoch 8/100
100/100 [==============================] - 0s 40us/step - loss: 0.2913
Epoch 9/100
100/100 [==============================] - 0s 40us/step - loss: 0.2883
Epoch 10/100
100/100 [==============================] - 0s 40us/step - loss: 0.2854
Epoch 11/100
100/100 [==============================] - 0s 50us/step - loss: 0.2829
Epoch 12/100
100/100 [==============================] - 0s 50us/step - loss: 0.2805
Ep

In [32]:
import json
import csv
with open('test_model_config.json', 'w') as modelfile:
    json.dump(model.get_config(), modelfile)

In [33]:
def rows_of(array):
    if len(array.shape) > 1:
        for n in range(array.shape[0]):
            yield (n, array[n])
    else:
        yield (0, array)

def open_csv(fname, column_headers=None):
    csvfile = open(fname, 'w', newline='')
    csvwriter = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    if !(column_headers is None):
        csvwriter.writerow(column_headers)
    return (csvfile, csvwriter)

def write_csv_matrix(csvwriter, matrix_array, precolumns):
    for (row_num, row_array) in rows_of(matrix_array):
        if len(matrix_array.shape) > 1:
            row_count = matrix_array.shape[0]
            col_count = matrix_array.shape[1]
        else:
            row_count = 1
            col_count = matrix_array.shape[0]
        csv_row = precolumns + [ row_count, col_count, row_num] + list(row_array)
        csvwriter.writerow(csv_row)

(csvfile, csvwriter) = open_csv('tst_model_weights.csv', ['a','b','c','d','e'])
for layer in model.layers:
    for weight_variable in layer.weights:
        weight_values = K.batch_get_value(weight_variable)
        write_csv_matrix(csvwriter, weight_values, [])
        
with open('test_model_weights.csv', 'w', newline='') as csvfile:
    weightswriter = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    for layer in model.layers:
        for weight_variable in layer.weights:
            weight_values = K.batch_get_value(weight_variable)
            for (row_num, row_array) in rows_of(weight_values):
                if len(weight_values.shape) > 1:
                    row_count = weight_values.shape[0]
                    col_count = weight_values.shape[1]
                else:
                    row_count = 1
                    col_count = weight_values.shape[0]                    
                csv_row = [layer.name, weight_variable.name,
                           row_count, col_count, row_num] + list(row_array)
                weightswriter.writerow(csv_row)

In [63]:
def write_matrix_csv(fname, matrix_gen):
    with open(fname, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
        for (precolumns, matrix_array) in matrix_gen:
            for (n, row_array) in rows_of(matrix_array):
                if len(matrix_array.shape) > 1:
                    rows, columns = matrix_array.shape
                else:
                    rows, columns = 1, matrix_array.shape[0]
                csvwriter.writerow(precolumns + [rows, columns, n] + list(row_array))

In [64]:
def layer_weights(model):
    for layer in model.layers:
        for weight_variable in layer.weights:
            weight_values = K.batch_get_value(weight_variable)
            yield ([layer.name, weight_variable.name], weight_values)            

In [65]:
write_matrix_csv('test.csv', layer_weights(model))

In [68]:
input = np.array([[0.,0.,0.],[0.,0.,1.],[0.,1.,0.],[1.,0.,0.]])
output = model.predict(input)
write_matrix_csv('predict.txt', [(['in'], input), (['out'], output)])